# Market Sentiment Analysis Pipeline

Building a system to analyze market sentiment for any company. 

In [ ]:
# Getting all the packages we need
!pip install langchain langchain-google-vertexai langchain-community mlflow requests beautifulsoup4 pydantic python-dotenv exa-py pandas -q

In [ ]:
import os
import json
import time
import requests
import re
from datetime import datetime, timedelta
from typing import Dict, Any, List, Optional
from pydantic import BaseModel, Field, validator

# langchain stuff
from langchain.tools import Tool
from langchain.prompts import PromptTemplate
from langchain_google_vertexai import ChatVertexAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.output_parsers import PydanticOutputParser

import mlflow
from mlflow.tracking import MlflowClient

# google cloud setup
import vertexai
from google.cloud import aiplatform

PROJECT_ID = "your-project-id"  
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)
aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# what we want back from the analysis
class SentimentProfile(BaseModel):
    company_name: str = Field(description="the company we analyzed")
    stock_code: str = Field(description="ticker symbol like AAPL")
    newsdesc: str = Field(description="what the news was about")
    sentiment: str = Field(description="positive/negative/neutral")
    people_names: List[str] = Field(description="executives, analysts mentioned")
    places_names: List[str] = Field(description="countries, cities in the news")
    other_companies_referred: List[str] = Field(description="competitors or partners mentioned")
    related_industries: List[str] = Field(description="sectors this affects")
    market_implications: str = Field(description="what this means for the stock")
    confidence_score: float = Field(description="how sure we are, 0 to 1")
    
    @validator('sentiment')
    def check_sentiment(cls, v):
        allowed = ['Positive', 'Negative', 'Neutral']
        if v not in allowed:
            raise ValueError(f'Sentiment must be one of {allowed}')
        return v
    
    @validator('confidence_score')
    def check_confidence(cls, v):
        if not 0.0 <= v <= 1.0:
            raise ValueError('Confidence score should be between 0 and 1')
        return v

In [ ]:
class StockCodeExtractor:
    def __init__(self):
        self.llm = ChatVertexAI(
            model_name="gemini-2.0-flash-exp",
            temperature=0.1,
            max_tokens=50
        )
        
        # hardcoded common ones to save API calls
        self.known_stocks = {
            "apple inc": "AAPL", "apple": "AAPL",
            "microsoft corporation": "MSFT", "microsoft": "MSFT",
            "google": "GOOGL", "alphabet inc": "GOOGL", "alphabet": "GOOGL",
            "amazon.com inc": "AMZN", "amazon": "AMZN",
            "tesla inc": "TSLA", "tesla": "TSLA",
            "meta platforms inc": "META", "facebook": "META", "meta": "META",
            "nvidia corporation": "NVDA", "nvidia": "NVDA"
        }
        
        self.prompt = PromptTemplate.from_template(
            "What's the stock ticker for: {company_name}\n"
            "Just give me the ticker symbol (like AAPL for Apple):"
        )
    
    def extract(self, company_name: str) -> Dict[str, Any]:
        # check our lookup table first
        name_clean = company_name.lower().strip()
        if name_clean in self.known_stocks:
            return {"stock_code": self.known_stocks[name_clean], "method": "lookup"}
        
        # try yahoo finance
        try:
            url = "https://query1.finance.yahoo.com/v1/finance/search"
            resp = requests.get(url, params={'q': company_name}, timeout=10)
            if resp.status_code == 200:
                data = resp.json()
                for quote in data.get('quotes', []):
                    if quote.get('typeDisp') in ['Equity', 'Stock']:
                        return {"stock_code": quote.get('symbol'), "method": "yahoo_api"}
        except Exception as e:
            # yahoo can be flaky sometimes
            pass
        
        # ask the LLM as last resort
        try:
            chain = self.prompt | self.llm | StrOutputParser()
            result = chain.invoke({"company_name": company_name})
            ticker = result.strip().upper()
            # basic validation - tickers are usually 1-5 letters
            if re.match(r'^[A-Z]{1,5}$', ticker):
                return {"stock_code": ticker, "method": "ai_guess"}
        except Exception as e:
            # sometimes the AI fails too
            pass
        
        return {"stock_code": None, "method": "gave_up"}

In [ ]:
class NewsFetcher:
    def __init__(self):
        self.max_articles = 8  # don't want to overwhelm the AI
        
    def fetch_news(self, company_name: str, stock_code: str) -> List[Dict[str, Any]]:
        articles = []
        
        # try to get news from yahoo finance
        try:
            url = "https://query1.finance.yahoo.com/v1/finance/search"
            resp = requests.get(url, params={'q': stock_code, 'newsCount': 8}, timeout=10)
            if resp.status_code == 200:
                data = resp.json()
                for item in data.get('news', []):
                    articles.append({
                        "title": item.get('title', ''),
                        "content": item.get('summary', '')[:500],  # truncate long summaries
                        "source": "yahoo_finance"
                    })
        except Exception as e:
            # yahoo API can be unreliable
            print(f"Couldn't fetch from Yahoo: {e}")
        
        # fallback if we got nothing
        if not articles:
            # better than crashing
            articles = [{
                "title": f"Market Update: {company_name}",
                "content": f"Recent business developments and market activity for {company_name} ({stock_code}). Check financial news sources for latest updates.",
                "source": "fallback"
            }]
        
        return articles[:self.max_articles]

In [ ]:
class SentimentAnalyzer:
    def __init__(self):
        self.llm = ChatVertexAI(
            model_name="gemini-2.0-flash-exp",
            temperature=0.2,  # some creativity but not too wild
            max_tokens=2000
        )
        
        self.parser = PydanticOutputParser(pydantic_object=SentimentProfile)
        
        # the main prompt for analysis
        self.prompt = PromptTemplate(
            template="""Look at these news articles about {company_name} ({stock_code}) and tell me what you think:

{news_content}

I need to know:
- Is the overall vibe positive, negative, or neutral?
- Who are the key people mentioned? (CEOs, analysts, etc.)
- What places come up? (cities, countries, markets)
- Any other companies mentioned?
- What industries does this affect?
- What does this mean for the stock price?
- How confident are you? (give me a number from 0 to 1)

{format_instructions}

Response:""",
            input_variables=["company_name", "stock_code", "news_content"],
            partial_variables={"format_instructions": self.parser.get_format_instructions()}
        )
    
    def analyze(self, company_name: str, stock_code: str, articles: List[Dict[str, Any]]) -> Dict[str, Any]:
        # combine all the news into one block
        news_text = "\n\n".join([
            f"Headline: {article['title']}\nSummary: {article['content']}"
            for article in articles
        ])
        
        try:
            chain = self.prompt | self.llm | self.parser
            result = chain.invoke({
                "company_name": company_name,
                "stock_code": stock_code,
                "news_content": news_text
            })
            
            return result.dict()
            
        except Exception as e:
            print(f"Analysis failed: {e}")
            # return something reasonable instead of crashing
            return {
                "company_name": company_name,
                "stock_code": stock_code,
                "newsdesc": f"Couldn't properly analyze news for {company_name} - check the logs",
                "sentiment": "Neutral",
                "people_names": [],
                "places_names": [],
                "other_companies_referred": [],
                "related_industries": [],
                "market_implications": "Analysis failed - unable to determine impact",
                "confidence_score": 0.1  # very low confidence when we fail
            }

In [ ]:
class MLflowTracker:
    def __init__(self, experiment_name: str = "sentiment-analysis"):
        mlflow.set_tracking_uri("file:./mlruns")
        try:
            mlflow.create_experiment(experiment_name)
        except:
            pass  # already exists probably
        mlflow.set_experiment(experiment_name)
    
    def log_step(self, step_name: str, metrics: Dict[str, Any], params: Dict[str, Any] = None):
        # log parameters if we have any
        if params:
            for key, value in params.items():
                mlflow.log_param(f"{step_name}_{key}", value)
        
        # log metrics and tags
        for key, value in metrics.items():
            if isinstance(value, (int, float)):
                mlflow.log_metric(f"{step_name}_{key}", value)
            else:
                mlflow.set_tag(f"{step_name}_{key}", str(value))
    
    def log_result(self, result: Dict[str, Any]):
        # save the important stuff
        mlflow.log_metric("confidence", result.get("confidence_score", 0))
        mlflow.set_tag("final_sentiment", result.get("sentiment", "Unknown"))
        mlflow.set_tag("ticker", result.get("stock_code", "Unknown"))
        
        # save full result as file
        result_file = "analysis_result.json"
        with open(result_file, "w") as f:
            json.dump(result, f, indent=2)

        mlflow.log_artifact(result_file)            os.remove(result_file)

        # cleanup temp file        if os.path.exists(result_file):

In [ ]:
class MarketSentimentPipeline:
    def __init__(self):
        # wire up all the pieces
        self.stock_extractor = StockCodeExtractor()
        self.news_fetcher = NewsFetcher()
        self.sentiment_analyzer = SentimentAnalyzer()
        self.tracker = MLflowTracker()
    
    def analyze(self, company_name: str) -> Dict[str, Any]:
        with mlflow.start_run():
            start_time = time.time()
            
            # first, figure out the stock ticker
            print(f"Looking up ticker for {company_name}...")
            stock_result = self.stock_extractor.extract(company_name)
            ticker = stock_result.get("stock_code")
            
            if not ticker:
                error_msg = f"Couldn't find ticker for {company_name}"
                print(error_msg)
                return {"error": error_msg, "company_name": company_name}
            
            print(f"Found ticker: {ticker}")
            self.tracker.log_step("ticker_lookup", 
                {"success": 1, "method": stock_result["method"]},
                {"company": company_name, "ticker": ticker})
            
            # grab some recent news
            print(f"Getting news for {ticker}...")
            articles = self.news_fetcher.fetch_news(company_name, ticker)
            print(f"Found {len(articles)} articles")
            
            self.tracker.log_step("news_fetch",
                {"article_count": len(articles)},
                {"ticker": ticker})
            
            # run the sentiment analysis
            print("Running sentiment analysis...")
            result = self.sentiment_analyzer.analyze(company_name, ticker, articles)
            
            # add some metadata
            result["execution_time"] = round(time.time() - start_time, 2)
            result["timestamp"] = datetime.now().isoformat()
            
            self.tracker.log_result(result)
            print(f"Done! Sentiment: {result.get('sentiment', 'Unknown')}")
            

            return resultpipeline = MarketSentimentPipeline()

# set up the pipeline

## Let's test this thing

Ok the pipeline should be working now. Just call `pipeline.analyze("Company Name")` and it will:

1. Find the stock ticker
2. Pull some recent news
3. Figure out if the sentiment is positive/negative/neutral
4. Extract mentions of people, places, other companies
5. Log everything for tracking

Try it out:
```python
result = pipeline.analyze("Apple")
print(json.dumps(result, indent=2))
```

In [ ]:
# let's try it with Google
print("Testing with Google...")

result = pipeline.analyze("Google")
print(json.dumps(result, indent=2))

In [ ]:
# try a few more companies
companies_to_test = ["Apple", "Microsoft", "Tesla"]

all_results = {}
for company in companies_to_test:
    print(f"\nTrying {company}...")
    try:
        result = pipeline.analyze(company)
        all_results[company] = result
        sentiment = result.get('sentiment', 'Unknown')
        confidence = result.get('confidence_score', 0)
        print(f"✓ {company}: {sentiment} (confidence: {confidence:.2f})")
    except Exception as e:
        print(f"✗ {company} failed: {e}")
        all_results[company] = {"error": str(e)}

# dump everything to a file
with open("test_results.json", "w") as f:
    json.dump(all_results, f, indent=2)

print(f"\nSaved all results to test_results.json")

## What the output looks like

You'll get back a JSON object like this:

```json
{
  "company_name": "Google",
  "stock_code": "GOOGL",
  "newsdesc": "Recent news about Google's earnings and AI developments",
  "sentiment": "Positive",
  "people_names": ["Sundar Pichai", "Ruth Porat"],
  "places_names": ["California", "United States"],
  "other_companies_referred": ["Microsoft", "Apple"],
  "related_industries": ["Technology", "Cloud Computing", "AI"],
  "market_implications": "Strong earnings and AI progress look good for the stock",
  "confidence_score": 0.85,
  "execution_time": 12.34,
  "timestamp": "2025-09-25T10:30:00"
}
```

## MLflow tracking

Everything gets logged to MLflow so you can:
- Compare different runs
- Track performance over time
- Debug when things go wrong

Run `mlflow ui` to see the web interface.

In [ ]:
# helper for running multiple companies at once
def batch_analyze(companies: List[str]) -> Dict[str, Dict[str, Any]]:
    results = {}
    for company in companies:
        print(f"Working on {company}...")
        try:
            result = pipeline.analyze(company)
            results[company] = result
        except Exception as e:
            print(f"Failed on {company}: {e}")
            results[company] = {"error": str(e), "company_name": company}
    return results

# make this into a langchain tool if needed
def make_langchain_tool():
    def wrapper(company_name: str) -> str:
        result = pipeline.analyze(company_name)
        return json.dumps(result, indent=2)
    
    return Tool(
        name="sentiment_analyzer",
        description="Analyzes market sentiment for any company - returns JSON with sentiment, key people, market impact, etc.",
        func=wrapper
    )

# create the tool
tool = make_langchain_tool()
print(f"Created tool: {tool.name}")
print(f"Description: {tool.description}")